### Defining a state

State is the memory shared between all the nodes in the graph. It is used to store information that needs to be accessed or modified by multiple nodes during the execution of the graph.

In [78]:
from typing import TypedDict, List
class State(TypedDict):
    topic :str
    raw_sources:List[str]
    summary_points:List[str]
    tags:List[str]

### Creating a search_node

- Node: Its a full blown task. (state -> process -> updated state) creates a node.
- Edge: Transition from one node to another node.

In [79]:
def search_node(state: State) -> State:
    topic = state["topic"]
    # TODO: replace this with MCP search_web later
    fake_results = [
        f"Intro article about {topic}",
        f"Recent blog post on {topic}",
        f"Technical overview of {topic}",
    ]
    return {**state, "raw_sources": fake_results}


In [80]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(State)
builder.add_node("search",search_node)
builder.add_edge(START,"search")
builder.add_edge("search",END)
graph = builder.compile()


result = graph.invoke({"topic":"Transformer Architecture"})

In [81]:
result

{'topic': 'Transformer Architecture',
 'raw_sources': ['Intro article about Transformer Architecture',
  'Recent blog post on Transformer Architecture',
  'Technical overview of Transformer Architecture']}

### Creating a summarize_node

In [82]:
def summarize_node(state:State) -> State:
    raw_sources = state["raw_sources"]
    fake_summary_points = [
        "It is fast",
        "It is quick",
        "It is efficient"
    ]
    return {**state,"summary_points":fake_summary_points}

In [83]:
builder.add_node("summarize",summarize_node)
builder.add_edge("search","summarize")
builder.add_edge("summarize",END)
graph = builder.compile()

Adding a node to a graph that has already been compiled. This will not be reflected in the compiled graph.
Adding an edge to a graph that has already been compiled. This will not be reflected in the compiled graph.
Adding an edge to a graph that has already been compiled. This will not be reflected in the compiled graph.


In [84]:
out = graph.invoke({
    "topic":"transformer architecture"
})

In [85]:
out["summary_points"]

['It is fast', 'It is quick', 'It is efficient']

In [86]:
def tag_node(state:State) -> State:
    summary = state["summary_points"]
    fake_tags = [
        "research",
        "summary",
        "notes"
    ]
    return {**state,"tags":fake_tags}

In [87]:
builder.add_node("tag",tag_node)
builder.add_edge("summarize","tag")
builder.add_edge("tag",END)
graph = builder.compile()

Adding a node to a graph that has already been compiled. This will not be reflected in the compiled graph.
Adding an edge to a graph that has already been compiled. This will not be reflected in the compiled graph.
Adding an edge to a graph that has already been compiled. This will not be reflected in the compiled graph.


In [88]:
out = graph.invoke({
    "topic":"transformer architecture"
})

In [89]:
out["tags"]

['research', 'summary', 'notes']

In [90]:
out

{'topic': 'transformer architecture',
 'raw_sources': ['Intro article about transformer architecture',
  'Recent blog post on transformer architecture',
  'Technical overview of transformer architecture'],
 'summary_points': ['It is fast', 'It is quick', 'It is efficient'],
 'tags': ['research', 'summary', 'notes']}

In [91]:
def store_node(state: State) -> State:
    topic = state['topic']
    summary_points = state['summary_points']
    tags = state['tags']

    note_content = f"Topic: {topic}\n"
    note_content += "Summary:\n "+"\n".join(f"- {p}" for p in summary_points)
    note_content += f"\nTags: {', '.join(tags)}"

    print(f"\nNote:\n{note_content}")

    return state

In [92]:
builder.add_node("store",store_node)
builder.add_edge("tag","store")
builder.add_edge("store",END)
graph = builder.compile()

Adding a node to a graph that has already been compiled. This will not be reflected in the compiled graph.
Adding an edge to a graph that has already been compiled. This will not be reflected in the compiled graph.
Adding an edge to a graph that has already been compiled. This will not be reflected in the compiled graph.


In [93]:
out = graph.invoke({
    "topic" : "transformer architecture"
})
out


Note:
Topic: transformer architecture
Summary:
 - It is fast
- It is quick
- It is efficient
Tags: research, summary, notes


{'topic': 'transformer architecture',
 'raw_sources': ['Intro article about transformer architecture',
  'Recent blog post on transformer architecture',
  'Technical overview of transformer architecture'],
 'summary_points': ['It is fast', 'It is quick', 'It is efficient'],
 'tags': ['research', 'summary', 'notes']}